In [1]:
import pandas
import pandas as pd

from GA import GA
import optproblems.cec2005 as optproblems

# Test Functions
- F2: Shifted Schwefel’s Problem 1.2
    - Uni-modal without noise
- F4: Shifted Schwefel’s Problem 1.2 with Noise in Fitness
    - Uni-modal with noise
- F8: Shifted Rotated Ackley’s Function with Global Optimum on Bounds
    - Single function
    - Global optimum on bounds
    - Multi-modal
- F13: Shifted Expanded Griewank’s plus Rosenbrock’s Function (F8F2)
    - Expanded Function
    - Multi-modal
- F17: Rotated Version of Hybrid Composition Function with Noise in Fitness
    - Gaussian noise in fitness
    - Multi-modal
    - Huge number of local optima
    - In-deterministic
- F24: Rotated Hybrid Composition Function
    - Composite function
    - Huge number of local optima
    - Multi-modal

# 1st Iteration of Configurations - Experiment with GA with Different Hyper Parameters
First we explore how GA evaluates with a set of configurations using only lower numbers of generations

Hyper parameters

In [2]:
# F2 :   Unimodal without noise
# F4 :   Unimodal with noise
# F8 :   Single Function, Global Optimum on Bounds, Multi-modal
# F13:   Expanded Function, Multi-modal
# F17:   Gaussian noise in Fitness, Multi-modal, Huge number of local optima, indeterministic
# F24:   Composite Function, Huge number of local optima, Multi-modal

benchmarks_and_bounds = [optproblems.F2, [-100,100], "F2", optproblems.F4, [-100,100], "F4", optproblems.F8, [-32,32], "F8", optproblems.F13, [-3,1], "F13", optproblems.F17, [-5,5], "F17", optproblems.F24, [-5,5], "F24"]
no_dimensions = [10, 30, 50]
pop_sizes = [5, 10, 15]
num_generations = [50, 150]
tournament_t = [2, 4]
crossover_percentage = [0.5, 0.95, 1]
mutation_rates = [0.2, 0.8, 1]
decreasing_mutation_rate = [True, False]
elites = [0, 1]

average_over_range = 5

Run configuration suite over each configuration combination.

In [3]:
run_configuration_suite = False
if run_configuration_suite:

    list_for_df = []
    for i in range(0, len(benchmarks_and_bounds), 3):
        number_of_functions = (len(benchmarks_and_bounds)/3)
        print(f"Progress: {i/3} / {number_of_functions}")
        bound = benchmarks_and_bounds[i+1]
        benchmark_name = benchmarks_and_bounds[i+2]
        for no_dimension in no_dimensions:
            print(f"\tDimension: {no_dimension}")
            benchmark = benchmarks_and_bounds[i](no_dimension)
            optimal_solution = benchmark.get_optimal_solutions()
            benchmark.evaluate(optimal_solution[0])
            optimal_minimum = optimal_solution[0].objective_values
            for pop_size in pop_sizes:
                print(f"\t\tPopulation: {pop_size}")
                for t in tournament_t:
                    for crossover in crossover_percentage:
                        for mutation in mutation_rates:
                            for dmr in decreasing_mutation_rate:
                                for elite in elites:
                                    for generations in num_generations:
                                        average_fitness = 0
                                        average_difference_from_optimal = 0

                                        best_fitness = 0
                                        best_time = 0
                                        best_difference_from_optimal = 0

                                        for j in range(average_over_range):
                                            algorithm = GA(benchmark, no_dimension, bound, pop_size, num_generations=generations, t=t, crossover_percentage=crossover, mutation_rate=mutation, decreasing_mutation_rate=dmr, elite=elite)
                                            result, time = algorithm.run()
                                            fitness = result.fitness
                                            diff_from_optimal = fitness - optimal_minimum

                                            average_fitness += 1/average_over_range * fitness
                                            average_difference_from_optimal += 1/average_over_range * diff_from_optimal

                                            if j == 0:
                                                best_fitness = fitness
                                                best_difference_from_optimal = diff_from_optimal
                                                best_time = time
                                                continue

                                            if fitness < best_fitness:
                                                best_fitness = fitness

                                            if diff_from_optimal < best_difference_from_optimal:
                                                best_difference_from_optimal = diff_from_optimal

                                            if time < best_time:
                                                best_time = time

                                        result_and_configuration = [benchmark_name, optimal_minimum, best_fitness, best_difference_from_optimal, average_fitness, average_difference_from_optimal, best_time, generations, no_dimension, pop_size, t, crossover, mutation, dmr, elite, bound, average_over_range]
                                        list_for_df.append(result_and_configuration)

    labels = ["TestFunction", "OptimalMinimum", "BestFitness", "BestDiffFromOpt", "AverageFitness", "AverageDiffFromOpt", "TimePerRun", "Generations", "NoDimension", "PopSize", "TForTournament", "CrossoverPerc", "MutationRate", "DecreasingMutationRate", "Elite", "Bounds", "AverageOverRange"]
    df = pandas.DataFrame(data=list_for_df, columns=labels)
    df.to_pickle("GA_df.pkl")
else:
    df = pd.read_pickle("GA_df.pkl")

## Results of the 1st Iteration

# F2:

- Best Diff: 312.11 -> 150 Generation / 10 Dimension / 15 Pop Size / 2 T / 100% CR / 20% MR / DMR True / 1 Elite
    -  Error: 312.11 / 450 -> 69.36%
- Avr. Diff: 913.20 -> 150 Generation / 10 Dimension / 15 Pop Size / 4 T / 95%  CR / 20% MR / DMR True / 1 Elite
    - Error: 913.20 / 450 -> 202.93%

In [4]:
# Best Fitness
df_F2_1st = df.query("TestFunction == 'F2'")
df_F2_1st[df_F2_1st.BestDiffFromOpt == df_F2_1st.BestDiffFromOpt.min()]

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
339,F2,-450.0,-137.88671,312.11329,1386.667757,1836.667757,0.07543,150,10,15,2,1.0,0.2,True,1,"[-100, 100]",5


In [5]:
# Best Average Fitness
df_F2_1st[df_F2_1st.AverageDiffFromOpt == df_F2_1st.AverageDiffFromOpt.min()]

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
387,F2,-450.0,136.349155,586.349155,463.202679,913.202679,0.07794,150,10,15,4,0.95,0.2,True,1,"[-100, 100]",5


In [6]:
df_F2_1st

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
0,F2,-450.0,17690.525955,18140.525955,3.213329e+04,3.258329e+04,0.011128,50,10,5,2,0.5,0.2,True,0,"[-100, 100]",5
1,F2,-450.0,14235.020676,14685.020676,2.238459e+04,2.283459e+04,0.027806,150,10,5,2,0.5,0.2,True,0,"[-100, 100]",5
2,F2,-450.0,6013.072511,6463.072511,1.487255e+04,1.532255e+04,0.010348,50,10,5,2,0.5,0.2,True,1,"[-100, 100]",5
3,F2,-450.0,1868.742268,2318.742268,5.054819e+03,5.504819e+03,0.031108,150,10,5,2,0.5,0.2,True,1,"[-100, 100]",5
4,F2,-450.0,21611.965380,22061.965380,5.134586e+04,5.179586e+04,0.009959,50,10,5,2,0.5,0.2,False,0,"[-100, 100]",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1291,F2,-450.0,112341.304928,112791.304928,1.461942e+05,1.466442e+05,0.321514,150,50,15,4,1.0,1.0,True,1,"[-100, 100]",5
1292,F2,-450.0,590805.599163,591255.599163,9.641332e+05,9.645832e+05,0.171331,50,50,15,4,1.0,1.0,False,0,"[-100, 100]",5
1293,F2,-450.0,575018.988130,575468.988130,1.290336e+06,1.290786e+06,0.513789,150,50,15,4,1.0,1.0,False,0,"[-100, 100]",5
1294,F2,-450.0,232243.327721,232693.327721,3.293918e+05,3.298418e+05,0.154155,50,50,15,4,1.0,1.0,False,1,"[-100, 100]",5


# F4
- Best Diff: 459.44 -> 150 Generation / 10 Dimension / 15 Pop Size / 4 T / 95% CR / 20% MR / DMR False / 1 Elite
    - Error: 459.44 / 450 ->  102.09%
- Avr. Diff: 1841.4 -> 150 Generation / 10 Dimension / 15 Pop Size / 2 T / 95%  CR / 20% MR / DMR True / 1 Elite
    - Error: 1841.4 / 450 ->  409.2%

In [7]:
# Best Fitness
df_F4_1st = df.query("TestFunction == 'F4'")
df_F4_1st[df_F4_1st.BestDiffFromOpt == df_F4_1st.BestDiffFromOpt.min()]

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
1687,F4,-450.0,9.440808,459.440808,2122.914216,2572.914216,0.089259,150,10,15,4,0.95,0.2,False,1,"[-100, 100]",5


In [8]:
# Best Average Fitness
df_F4_1st[df_F4_1st.AverageDiffFromOpt == df_F4_1st.AverageDiffFromOpt.min()]

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
1611,F4,-450.0,553.466922,1003.466922,1391.481705,1841.481705,0.079499,150,10,15,2,0.95,0.2,True,1,"[-100, 100]",5


In [9]:
df_F4_1st

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
1296,F4,-450.0,19648.328195,20098.328195,4.668801e+04,4.713801e+04,0.009338,50,10,5,2,0.5,0.2,True,0,"[-100, 100]",5
1297,F4,-450.0,43889.355029,44339.355029,5.329062e+04,5.374062e+04,0.028457,150,10,5,2,0.5,0.2,True,0,"[-100, 100]",5
1298,F4,-450.0,3226.647401,3676.647401,1.438899e+04,1.483899e+04,0.009620,50,10,5,2,0.5,0.2,True,1,"[-100, 100]",5
1299,F4,-450.0,1952.865879,2402.865879,6.795449e+03,7.245449e+03,0.032871,150,10,5,2,0.5,0.2,True,1,"[-100, 100]",5
1300,F4,-450.0,53798.825239,54248.825239,1.240192e+05,1.244692e+05,0.011265,50,10,5,2,0.5,0.2,False,0,"[-100, 100]",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2587,F4,-450.0,161980.060540,162430.060540,2.346768e+05,2.351268e+05,0.328651,150,50,15,4,1.0,1.0,True,1,"[-100, 100]",5
2588,F4,-450.0,404325.050245,404775.050245,1.207540e+06,1.207990e+06,0.172086,50,50,15,4,1.0,1.0,False,0,"[-100, 100]",5
2589,F4,-450.0,474857.028769,475307.028769,1.385936e+06,1.386386e+06,0.524369,150,50,15,4,1.0,1.0,False,0,"[-100, 100]",5
2590,F4,-450.0,367568.143286,368018.143286,5.447924e+05,5.452424e+05,0.155512,50,50,15,4,1.0,1.0,False,1,"[-100, 100]",5


# F8
- Best Diff: 20.17 -> 150 Generation / 10 Dimension / 15 Pop Size / 4 T / 100% CR / 80% MR / DMR True / 1 Elite
    - Error: 20.17 / 140 -> 14.4%
- Avr. Diff: 20.47 -> Same
    - Error: 20.47 / 140 -> 14.6%

In [10]:
# Best Fitness
df_F8_1st = df.query("TestFunction == 'F8'")
df_F8_1st[df_F8_1st.BestDiffFromOpt == df_F8_1st.BestDiffFromOpt.min()]

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
3011,F8,-140.0,-119.829659,20.170341,-119.524006,20.475994,0.116559,150,10,15,4,1.0,0.8,True,1,"[-32, 32]",5


In [11]:
# Best Average Fitness
df_F8_1st[df_F8_1st.AverageDiffFromOpt == df_F8_1st.AverageDiffFromOpt.min()]

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
3011,F8,-140.0,-119.829659,20.170341,-119.524006,20.475994,0.116559,150,10,15,4,1.0,0.8,True,1,"[-32, 32]",5


In [12]:
df_F8_1st

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
2592,F8,-140.0,-118.828332,21.171668,-118.601792,21.398208,0.011346,50,10,5,2,0.5,0.2,True,0,"[-32, 32]",5
2593,F8,-140.0,-118.780497,21.219503,-118.632335,21.367665,0.034004,150,10,5,2,0.5,0.2,True,0,"[-32, 32]",5
2594,F8,-140.0,-119.053467,20.946533,-118.934699,21.065301,0.015518,50,10,5,2,0.5,0.2,True,1,"[-32, 32]",5
2595,F8,-140.0,-119.263744,20.736256,-119.165199,20.834801,0.046569,150,10,5,2,0.5,0.2,True,1,"[-32, 32]",5
2596,F8,-140.0,-118.832553,21.167447,-118.571401,21.428599,0.017156,50,10,5,2,0.5,0.2,False,0,"[-32, 32]",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3883,F8,-140.0,-118.752831,21.247169,-118.683254,21.316746,0.383418,150,50,15,4,1.0,1.0,True,1,"[-32, 32]",5
3884,F8,-140.0,-118.536631,21.463369,-118.443720,21.556280,0.194784,50,50,15,4,1.0,1.0,False,0,"[-32, 32]",5
3885,F8,-140.0,-118.496857,21.503143,-118.436180,21.563820,0.586340,150,50,15,4,1.0,1.0,False,0,"[-32, 32]",5
3886,F8,-140.0,-118.684637,21.315363,-118.661321,21.338679,0.175834,50,50,15,4,1.0,1.0,False,1,"[-32, 32]",5


# F13
- Best Diff: 1.0829 -> 150 Generation / 10 Dimension / 15 Pop Size / 4 T / 100% CR / 20% MR / DMR True / 1 Elite
    - Error: 1.0829 / 130 ->  0.83%
- Avr. Diff: 2.9072 -> Same
    - Error: 1.5791 / 130 ->  1.21%


In [13]:
# Best Fitness
df_F13_1st = df.query("TestFunction == 'F13'")
df_F13_1st[df_F13_1st.BestDiffFromOpt == df_F13_1st.BestDiffFromOpt.min()]

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
411,F13,-130.0,-128.917016,1.082984,-128.420892,1.579108,0.130163,150,10,15,4,1.0,0.2,True,1,"[-3, 1]",5


In [14]:
# Best Average Fitness
df_F13_1st[df_F13_1st.AverageDiffFromOpt == df_F13_1st.AverageDiffFromOpt.min()]

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
411,F13,-130.0,-128.917016,1.082984,-128.420892,1.579108,0.130163,150,10,15,4,1.0,0.2,True,1,"[-3, 1]",5


In [15]:
df_F13_1st

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
0,F13,-130.0,-114.747967,15.252033,123.382008,253.382008,0.015493,50,10,5,2,0.5,0.2,True,0,"[-3, 1]",5
1,F13,-130.0,-112.484813,17.515187,-81.314126,48.685874,0.047154,150,10,5,2,0.5,0.2,True,0,"[-3, 1]",5
2,F13,-130.0,-119.482405,10.517595,-117.122011,12.877989,0.014168,50,10,5,2,0.5,0.2,True,1,"[-3, 1]",5
3,F13,-130.0,-127.111143,2.888857,-122.551732,7.448268,0.043210,150,10,5,2,0.5,0.2,True,1,"[-3, 1]",5
4,F13,-130.0,68.014676,198.014676,1145.731194,1275.731194,0.015538,50,10,5,2,0.5,0.2,False,0,"[-3, 1]",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1291,F13,-130.0,18.446939,148.446939,86.802009,216.802009,0.559082,150,50,15,4,1.0,1.0,True,1,"[-3, 1]",5
1292,F13,-130.0,3078.646003,3208.646003,5533.162406,5663.162406,0.246466,50,50,15,4,1.0,1.0,False,0,"[-3, 1]",5
1293,F13,-130.0,5464.382972,5594.382972,9402.327325,9532.327325,0.757659,150,50,15,4,1.0,1.0,False,0,"[-3, 1]",5
1294,F13,-130.0,1636.088684,1766.088684,2376.984034,2506.984034,0.231453,50,50,15,4,1.0,1.0,False,1,"[-3, 1]",5


# F17
- Best Diff: 177.54 -> 150 Generation / 10 Dimension / 15 Pop Size / 2 T / 50% CR / 80% MR / DMR True / 1 Elite
    - Error: 177.54 / 120 ->  147.95%
- Avr. Diff: 223.09 -> 150 Generation / 10 Dimension / 15 Pop Size / 4 T / 100% CR / 20% MR / DMR True / 1 Elite
    - Error: 223.09 / 120 ->  185.9%

In [ ]:
# Best Fitness
df_F17_1st = df.query("TestFunction == 'F17'")
df_F17_1st[df_F17_1st.BestDiffFromOpt == df_F17_1st.BestDiffFromOpt.min()]

In [ ]:
# Best Average Fitness
df_F17_1st[df_F17_1st.AverageDiffFromOpt == df_F17_1st.AverageDiffFromOpt.min()]

In [ ]:
df_F17_1st

# F24
- Best Diff: 242.76 -> 150 Generation / 10 Dimension / 15 Pop Size / 4 T / 95% CR / 20% MR / DMR False / 1 Elite
    - Error: 242.76 / 260 -> 93.36%
- Avr. Diff: 351.24 -> 150 Generation / 10 Dimension / 10 Pop Size / 4 T / 95% CR / 20% MR / DMR True / 1 Elite
    - Error: 351.24 / 260 -> 135.09%

In [ ]:
# Best Fitness
df_F24_1st = df.query("TestFunction == 'F24'")
df_F24_1st[df_F24_1st.BestDiffFromOpt == df_F24_1st.BestDiffFromOpt.min()]

In [ ]:
# Best Average Fitness
df_F24_1st[df_F24_1st.AverageDiffFromOpt == df_F24_1st.AverageDiffFromOpt.min()]

In [ ]:
df_F24_1st

# Conclusion:

The optimizer performed the best on F8 and F13. F13 error was the lowest with the best fitness 0.38% away from the optimal. F8's best fitness was 14.4% away from the optimal. While the F13 error is good F8 is a significantly larger from the optimal and can be improved. The GA performed poorly on the rest of the test functions. The best fitness errors are as follows:
- F2:  69.36%
- F4: 102.09%
- F17: 147.95%
- F24: 93.36%

F8 and F13 share a similarity with their best scores. Both test function used a similar test configuration to achieve the best scores. `150 Generation / 10 Dimension / 15 Pop Size / 4 T / 100% CR / DMR True / 1 Elite`. The only difference is that F8 used 80% Mutation Rate whereas for F13 the best score was achieved with 20%.

Each test function scored the best after 150 generations, so we decided to increase the number of generations to see how they improve. For the 2nd iteration we kept the configuration of the best algorithm settings. The configurations with low frequency from the bests weren't carried on to the 2nd iteration for the algorithms where the error was huge.

Frequency of configuration among best difference and average difference from optimal:
- Generation:
    - 50 -> 0
    - 150 -> 12
- Dimension:
    - 10 -> 12
    - 30 -> 0
    - 50 -> 0
- Population Size:
    - 5 -> 0
    - 10 -> 1
    - 15 -> 11
- Tournament Size:
    - 2 -> 3
    - 4 -> 9
- Crossover Rate:
    - 50% -> 1
    - 95% -> 5
    - 100% -> 6
- Mutation Rate:
    - 20% -> 9
    - 80% -> 3
    - 100% -> 0
- Decreasing Mutation Rate:
    - True -> 10
    - False -> 2
- Elites:
    - 0 -> 0
    - 1 -> 12

# 2nd Iteration of Configurations
Same algorithm was used as for the 1st iteration but different hyper-parameters. Increased the number of generations to 250, 500, 1000. The population size was increased too and some configurations weren't carried over from iteration 1, i.e. the number of dimension was reduced to only 10.

In [ ]:
benchmarks_and_bounds = [optproblems.F2, [-100,100], "F2", optproblems.F4, [-100,100], "F4", optproblems.F8, [-32,32], "F8", optproblems.F13, [-3,1], "F13", optproblems.F17, [-5,5], "F17", optproblems.F24, [-5,5], "F24"]
no_dimensions = [10]
num_generations = [250, 500, 1000]
pop_sizes = [10, 15, 20]
tournament_t = [2, 4]
crossover_percentage = [0.95, 1]
mutation_rates = [0.2, 0.8]
decreasing_mutation_rate = [True]
elites = [0, 1]

average_over_range = 5

In [ ]:
run_configuration_suite = False
if run_configuration_suite:

    list_for_df = []
    for i in range(0, len(benchmarks_and_bounds), 3):
        number_of_functions = (len(benchmarks_and_bounds)/3)
        print(f"Progress: {i/3} / {number_of_functions}")
        bound = benchmarks_and_bounds[i+1]
        benchmark_name = benchmarks_and_bounds[i+2]
        for no_dimension in no_dimensions:
            print(f"\tDimension: {no_dimension}")
            benchmark = benchmarks_and_bounds[i](no_dimension)
            optimal_solution = benchmark.get_optimal_solutions()
            benchmark.evaluate(optimal_solution[0])
            optimal_minimum = optimal_solution[0].objective_values
            for pop_size in pop_sizes:
                print(f"\t\tPopulation: {pop_size}")
                for t in tournament_t:
                    for crossover in crossover_percentage:
                        for mutation in mutation_rates:
                            for dmr in decreasing_mutation_rate:
                                for elite in elites:
                                    for generations in num_generations:
                                        average_fitness = 0
                                        average_difference_from_optimal = 0

                                        best_fitness = 0
                                        best_time = 0
                                        best_difference_from_optimal = 0

                                        for j in range(average_over_range):
                                            algorithm = GA(benchmark, no_dimension, bound, pop_size, num_generations=generations, t=t, crossover_percentage=crossover, mutation_rate=mutation, decreasing_mutation_rate=dmr, elite=elite)
                                            result, time = algorithm.run()
                                            fitness = result.fitness
                                            diff_from_optimal = fitness - optimal_minimum

                                            average_fitness += 1/average_over_range * fitness
                                            average_difference_from_optimal += 1/average_over_range * diff_from_optimal

                                            if j == 0:
                                                best_fitness = fitness
                                                best_difference_from_optimal = diff_from_optimal
                                                best_time = time
                                                continue

                                            if fitness < best_fitness:
                                                best_fitness = fitness

                                            if diff_from_optimal < best_difference_from_optimal:
                                                best_difference_from_optimal = diff_from_optimal

                                            if time < best_time:
                                                best_time = time

                                        result_and_configuration = [benchmark_name, optimal_minimum, best_fitness, best_difference_from_optimal, average_fitness, average_difference_from_optimal, best_time, generations, no_dimension, pop_size, t, crossover, mutation, dmr, elite, bound, average_over_range]
                                        list_for_df.append(result_and_configuration)

    labels = ["TestFunction", "OptimalMinimum", "BestFitness", "BestDiffFromOpt", "AverageFitness", "AverageDiffFromOpt", "TimePerRun", "Generations", "NoDimension", "PopSize", "TForTournament", "CrossoverPerc", "MutationRate", "DecreasingMutationRate", "Elite", "Bounds", "AverageOverRange"]
    df_II = pandas.DataFrame(data=list_for_df, columns=labels)
    df_II.to_pickle("GA_df_specific.pkl")
else:
    df_II = pd.read_pickle("GA_df_specific.pkl")

# Results of the 2nd Iteration

# F2
### For 1st Iteration:
- Best Diff: 312.11 -> 150 Generation / 10 Dimension / 15 Pop Size / 2 T / 100% CR / 20% MR / DMR True / 1 Elite
    -  Error: 312.11 / 450 -> 69.36%
- Avr. Diff: 913.20 -> 150 Generation / 10 Dimension / 15 Pop Size / 4 T / 95%  CR / 20% MR / DMR True / 1 Elite
    - Error: 913.20 / 450 -> 202.93%

### For 2nd Iteration:
- Best Diff: 33.482 -> 1000 Generation / 10 Dimension / 20 Pop Size / 4 T / 95% CR / 20% MR / DMR True / 0 Elite
    -  Error: 33.482 / 450 -> 7.4%
- Avr. Diff: 113.75 -> 1000 Generation / 10 Dimension / 20 Pop Size / 4 T / 95% CR / 20% MR / DMR True / 1 Elite
    - Error: 113.75 / 450 -> 25.27%

## Conclusion for F2:
- Significant improvement by increasing the generations from 150 -> 1000 and the population from 15 -> 20.
- Best diff error is ~10× less
- Avg diff error is ~8× less
- In the 2nd iteration the best fitness was achieved by 4 as tournament size and 95% for crossover rate instead of 2 and 100%.

In [ ]:
df_II.query("TestFunction == 'F2'").sort_values(by=["BestDiffFromOpt"])

In [ ]:
df_II.query("TestFunction == 'F2'").sort_values(by=["AverageDiffFromOpt"])

# F4
### For 1st Iteration
- Best Diff: 459.44 -> 150 Generation / 10 Dimension / 15 Pop Size / 4 T / 95% CR / 20% MR / DMR False / 1 Elite
    - Error: 459.44 / 450 ->  102.09%
- Avr. Diff: 1841.4 -> 150 Generation / 10 Dimension / 15 Pop Size / 2 T / 95%  CR / 20% MR / DMR True / 1 Elite
    - Error: 1841.4 / 450 ->  409.2%

### For 2nd Iteration:
- Best Diff: 56.63 -> 500 Generation / 10 Dimension / 15 Pop Size / 4 T / 95% CR / 20% MR / DMR True / 1 Elite
    - Error: 56.63 / 450 -> 12.58%
- Avr. Diff: 190.78 -> 1000 Generation / 10 Dimension / 20 Pop Size / 4 T / 100% CR / 20% MR / DMR True / 1 Elite
    - Error: 190.78 / 450 -> 42.39%

## Conclusion for F4:
- Significant change by increasing the generation.
- The best fitness was achieved with 500 generation instead of 1000 and the population size was same for both iterations.
- The best fitness parameters didn't change much. Decreasing MR was used and the generation was increased from 150 to 500. However, for the average fitness the parameters changed. Higher population (20), generation (1000), tournament size (4) and crossover rate (100%) was used.

In [ ]:
df_II.query("TestFunction == 'F4'").sort_values(by=["BestDiffFromOpt"])

In [ ]:
df_II.query("TestFunction == 'F4'").sort_values(by=["AverageDiffFromOpt"])

# F8
(2nd best fitness among the first iteration)
### For 1st Iteration:
- Best Diff: 20.17 -> 150 Generation / 10 Dimension / 15 Pop Size / 4 T / 100% CR / 80% MR / DMR True / 1 Elite
    - Error: 20.17 / 140 -> 14.4%
- Avr. Diff: 20.47 -> Same
    - Error: 20.47 / 140 -> 14.6%

### For 2nd Iteration:
- Best Diff: 20.29 -> 1000 Generation / 10 Dimension / 20 Pop Size / 4 T / 100% CR / 20% MR / DMR True / 0 Elite
    - Error: 20.29 / 140 -> 14.49%
- Avr. Diff: 20.41 -> 1000 Generation / 10 Dimension / 10 Pop Size / 4 T / 100% CR / 20% MR / DMR True / 0 Elite
    - Error: 20.41 / 140 -> 14.57%

## Conclusion for F8:
- Best fitness error slightly increased!!!
- Gen 500 had similar result for best fitness as 1000.
- Average fitness improved slightly.
- No significant changes achieved when running it on higher number, as for previous functions.
- Since best fitness was reached at the 150 generation we conclude that for F8 the optimizer works well as for other test functions it took 500, 1000 generations to reach similar results.


In [ ]:
df_II.query("TestFunction == 'F8'").sort_values(by=["BestDiffFromOpt"])

In [ ]:
df_II.query("TestFunction == 'F8'").sort_values(by=["AverageDiffFromOpt"])

# F13
(Best fitness among the first iteration)
### For 1st Iteration:
- Best Diff: 1.8799 -> 150 Generation / 10 Dimension / 15 Pop Size / 4 T / 95% CR / 20% MR / DMR True / 0 Elite
    - Error: 1.8799 / 130 ->  1.34%
- Avr. Diff: 2.9072 -> 150 Generation / 10 Dimension / 10 Pop Size / 2 T / 95% CR / 20% MR / DMR True / 1 Elite
    - Error: 2.9072 / 130 ->  2.28%

### For 2nd Iteration:
- Best Diff: 1.8799 -> 150 Generation / 10 Dimension / 15 Pop Size / 4 T / 95% CR / 20% MR / DMR True / 0 Elite
    - Error: 1.8799 / 130 ->  1.34%
- Avr. Diff: 2.9072 -> 150 Generation / 10 Dimension / 10 Pop Size / 2 T / 95% CR / 20% MR / DMR True / 1 Elite
    - Error: 2.9072 / 130 ->  2.28%

## Conclusion:


In [ ]:
df_II.query("TestFunction == 'F13'")

# F17
### For 1st Iteration:
- Best Diff: 177.54 -> 150 Generation / 10 Dimension / 15 Pop Size / 2 T / 50% CR / 80% MR / DMR True / 1 Elite
    - Error: 177.54 / 120 ->  147.95%
- Avr. Diff: 223.09 -> 150 Generation / 10 Dimension / 15 Pop Size / 4 T / 100% CR / 20% MR / DMR True / 1 Elite
    - Error: 223.09 / 120 ->  185.9%

### For 2nd Iteration:


## Conclusion:


In [ ]:
df_II.query("TestFunction == 'F17'")

# F24
### For 1st Iteration:
- Best Diff: 242.76 -> 150 Generation / 10 Dimension / 15 Pop Size / 4 T / 95% CR / 20% MR / DMR False / 1 Elite
    - Error: 242.76 / 260 -> 93.36%
- Avr. Diff: 351.24 -> 150 Generation / 10 Dimension / 10 Pop Size / 4 T / 95% CR / 20% MR / DMR True / 1 Elite
    - Error: 351.24 / 260 -> 135.09%

### For 2nd Iteration:


## Conclusion:


In [ ]:
df_II.query("TestFunction == 'F24'")